In [20]:
from transformers import T5ForConditionalGeneration, AutoTokenizer, T5Tokenizer
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import re
import numpy as np

In [2]:
model = T5ForConditionalGeneration.from_pretrained('./my_model/')


In [3]:
model_name = 'cointegrated/rut5-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

c:\Users\Den\envs\envcolab\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Den\.cache\huggingface\hub\models--cointegrated--rut5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/828k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\Den\envs\envcolab\Lib\site-packages\transformers\convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [42]:
def generate_tox(input_text, model=model, tokenizer=tokenizer):
    
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Generate the output
    output_ids = model.to('cpu').generate(input_ids, max_length=50)

    # Decode the output
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    def remove_text_inside_brackets(text):
        return re.sub(r'<[^>]*>', '', text)

    return remove_text_inside_brackets(decoded_output)

In [43]:
generate_tox("Привет, всем")

'блядь, привет всем!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

In [16]:
tokenizer_chat = T5Tokenizer.from_pretrained("cointegrated/rut5-small-chitchat")
model_chat = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-small-chitchat")

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

c:\Users\Den\envs\envcolab\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Den\.cache\huggingface\hub\models--cointegrated--rut5-small-chitchat. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


spiece.model:   0%|          | 0.00/640k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/259M [00:00<?, ?B/s]

In [35]:
def generate_text(prompt, model_chat = model_chat, tokenizer_chat = tokenizer_chat, max_length=50):
    
    inputs = tokenizer_chat(prompt, return_tensors='pt')
    with torch.no_grad():
        hypotheses = model_chat.generate(
            **inputs, 
            do_sample=True, top_p=0.5, num_return_sequences=3, 
            repetition_penalty=2.5,
            max_length=max_length,
        )

    
    choice_idx = np.random.randint(0, len(hypotheses))
    h = hypotheses[choice_idx]
    return tokenizer_chat.decode(h, skip_special_tokens=True)


    for h in hypotheses:
        print(tokenizer_chat.decode(h, skip_special_tokens=True))

In [44]:
# Example prompt in Russian
text = 'Привет! Расскажи, как твои дела?'

kek = generate_text(text)
kek

'Все хорошо.'

In [47]:
def msg_pipeline(promt):
    answer = generate_text(promt)
    tox_answer = generate_tox(answer)
    return tox_answer

In [48]:
msg_pipeline(text)

'блядь с ними все в порядке.'